In [8]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np
import os

### DEFINE

In [13]:
# 初始年月份
set_data_year = 2023 #start_year
set_data_month = 1  #start_month
end_year = 2023
end_month = 2

# 檔案位置
Site = './data/'

# 檔案輸出位置 (沒資料夾會自動新增)
custom_output_path_dfTime = './dfTime/TEST(不分車商)'
custom_output_path_dfTLC = './dfTLC/month(不分車商)'

#是否存檔 1存,0不存
dfTime_save = 1
dfTLC_save = 0


#-------------------------------------------------------------------------------------------------#
# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   

print("done時間:", datetime.datetime.now())

done時間: 2023-08-18 23:41:15.554128


### 節日

In [10]:
# from holidays.calendars import gregorian
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
                
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"

### 執行
    1年約10分鐘

In [14]:
# 記錄開始時間
start_time = time.time()

#年月設定
data_year = set_data_year
data_month = set_data_month

#停止共乘日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')


for i in range(len(datasite_fhvhv)):
###  留下需要特徵 & 移除不在當月資料
    # 建立一個空的DataFrame來存放合併後的數據
    df_fhvhv = pd.DataFrame()
    df_yellow = pd.DataFrame()
    df_TLC = pd.DataFrame()
    
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(datasite_fhvhv[i])
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)    

    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(datasite_yellow[i])
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)    

    ###  特徵整理,統一命名
    #名稱統一
    rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
    df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
    rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
    df_yellow.rename(columns=rename_dict_yellow, inplace=True)  
    #fhvhv_Name
    df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
    df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'
    #yellow 新增特徵 TripTime,Name
    df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
    df_yellow['Name'] = 'yellow'
    df_yellow['shared_match_flag'] = 'N'
    #yellow合併到fhvhv
    df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
    df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)        
    #清理暫存
    del df_yellow ,df_fhvhv
    
    ###移除非目標車商
    df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0004'].index)


    ###  移除上車地點 PULocationID 未知 (264,265)
    df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]


    ###  移除不符合的共乘資料
    # 定義起始日期和結束日期
    df_TLC = df_TLC[~ ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date)))]
    df_TLC = df_TLC[~ ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 )))]


    ###  移除 TripTime <= 0
    df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)


    ###  移除TripMiles <= 0
    df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)


    ### 增加所有節日標籤
    df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
    df_TLC['is_holiday'].value_counts()


    ###  新增星期特徵&年月日時切分
    df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
    df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
    df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
    df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
    df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
    df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

    #####移除Name
    df_TLC = df_TLC.drop(columns=['Name'], axis=1)
    
    ###  時間序列
    dfTime_TLC = df_TLC.groupby(['year','month','day',
                         'hour','weekday','PULocationID','is_holiday'
                      ]).size().reset_index(name="次數")

    ###  檔案輸出
    if dfTime_save > 0 :
        os.makedirs(os.path.dirname(f'{custom_output_path_dfTime}/dfTime_TLC-{data_year}-{data_month}.parquet'), exist_ok=True)
        dfTime_TLC.to_parquet(f'{custom_output_path_dfTime}/dfTime_TLC-{data_year}-{data_month}.parquet')
    if dfTLC_save > 0 :    
        os.makedirs(os.path.dirname(f'{custom_output_path_dfTLC}/TLC-{set_data_year}-{data_month}.parquet'), exist_ok=True)
        df_TLC.to_parquet(f'{custom_output_path_dfTLC}/TLC-{set_data_year}-{data_month}.parquet')
    #清理暫存
    del dfTime_TLC,df_TLC
    print(f"done:{data_year}-{data_month}")
    
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    

#清理暫存
del uber_start_date ,uber_end_date,lyft_start_date,lyft_end_date,lyft_start_date1,data_month,data_year

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

done:2023-1
done:2023-2

程式執行花費的時間： 145.23 秒
done時間: 2023-08-18 23:43:46.408285
